In [1]:
import pandas as pd
import numpy as np
import re
from os import listdir

# Data transformation from original file

Sheet names: 
- 'App - Usage Time'
- 'App - Usage Count'
- 'Web - Usage Time'
- 'Web - Usage Count'
- 'Device Unlocks'

In [19]:
raw_data = pd.read_excel('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/raw/StayFree Export - Total Usage - 6_24_23 (1).xls',
                         sheet_name='App - Usage Count').rename({'Unnamed: 0': ''}, axis=1)
raw_data.head(3)

,,Device,"October 19, 2022","October 20, 2022","October 21, 2022","October 22, 2022","October 23, 2022","October 24, 2022","October 25, 2022","October 26, 2022",...,"June 16, 2023","June 17, 2023","June 18, 2023","June 19, 2023","June 20, 2023","June 21, 2023","June 22, 2023","June 23, 2023","June 24, 2023",Total Usage
0,Amazon Shopping,motorola moto g(6) plus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.0
1,Android System,motorola moto g(6) plus,1.0,2.0,0.0,5.0,1.0,4.0,0.0,0.0,...,1.0,0.0,2.0,0.0,3.0,2.0,0.0,1.0,2.0,703.0
2,Biblioapp UdeA,motorola moto g(6) plus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0


Dataframe transformation

In [20]:
# Drop unnecesary columns
raw_data = raw_data.drop(columns=['Total Usage', 'Device'])
# Eliminate the last 4 rows of the data
raw_data = raw_data.iloc[:-4]
# Transpose the dataframe
raw_data = raw_data.T
# Set the first row as column names
raw_data.columns = raw_data.iloc[0]
# Eliminating duplicated columns
duplicated_columns = raw_data.columns[raw_data.columns.duplicated()]
raw_data = raw_data.loc[:, ~raw_data.columns.duplicated()]
# Drop the first row since it is now the header
raw_data = raw_data[1:].reset_index().rename({'index': 'date'}, axis=1)
# Time transformation to minutes, (only time data, not count data)
for col in raw_data.columns:
    if col != 'date':
        raw_data[col] = round(pd.to_timedelta(raw_data[col]).dt.total_seconds() / 60, 2)
    else: pass

In [21]:
raw_data

,date,Amazon Shopping,Android System,Biblioapp UdeA,Bing,Calculator,Calendar,Call Management,Camera,CamScanner,...,Twitch,Twitter,tyba,Upwork Talent,VpnDialogs,Wallpapers,WhatsApp,YouTube,YouTube Music,Zoom
0,"October 19, 2022",0.0,1.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
1,"October 20, 2022",0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,82.0,5.0,0.0,0.0
2,"October 21, 2022",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,61.0,0.0,0.0,0.0
3,"October 22, 2022",0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,168.0,8.0,0.0,0.0
4,"October 23, 2022",0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,98.0,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,"June 20, 2023",0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,37.0,0.0
245,"June 21, 2023",0.0,2.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,...,0.0,0.0,2.0,3.0,0.0,0.0,122.0,4.0,23.0,0.0
246,"June 22, 2023",0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,57.0,2.0,1.0,0.0
247,"June 23, 2023",0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,131.0,24.0,35.0,0.0


# Testing

In [66]:
data = pd.read_csv('D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/web_usage_count_motorola.csv',
                   delimiter=';',
                   parse_dates=['date'])
data.head(2)

,date,udea.edu.co,chat.openai.com,tiktok.com,vm.tiktok.com,openai.com,google.com,instagram.com
0,2022-10-28,2,0,2,2,0,12,0
1,2022-10-30,1,0,10,3,0,5,0


Function to filter the dataframe based on the values we want

In [30]:
def transform_dataframe(dataframe: pd.DataFrame, values_greater_than):
    data_copy = dataframe.copy()

    #packing the name of columns and sums in a dictionary to use in further codes
    dictionary_sums = {}
    for column in data_copy:
        if (column != 'date') and \
           (sum(data_copy[column]) >= values_greater_than):
            
            dictionary_sums[column] = sum(data_copy[column])

    names_of_columns_to_stay = [column_name for column_name in dictionary_sums.keys()]
    names_of_columns_to_stay.insert(0, 'date')

    #filtering dataframe
    transformed_dataframe = data_copy[names_of_columns_to_stay]
    
    return transformed_dataframe

# Transformation of the actual data

In [5]:
path = "D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed"
dir_list = listdir(path)
dir_list

['.gitkeep',
 'app_usage_count_motorola.csv',
 'app_usage_time_motorola.csv',
 'device_unlocks.csv',
 'web_usage_count_chrome.csv',
 'web_usage_count_edge.csv',
 'web_usage_count_motorola.csv',
 'web_usage_time_chrome.csv',
 'web_usage_time_edge.csv',
 'web_usage_time_motorola.csv']

If you want to transform files with a specified word in it, replace it in the fist line of the code.
- For "count" files, the delimiter should be changed to ";"
- For "time" files, the delimiter should be changed to ","

In [77]:
word_to_find = "count" # word in the name of the file that you want to search for
for file in dir_list:
    word_is_in = re.search(word_to_find, file)
    if word_is_in:
        file_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/{file}'
        data = pd.read_csv(file_path,
                           delimiter=';',
                           parse_dates=['date'])
        
        data_transformed = transform_dataframe(data, 100)
        data_transformed = data_transformed.set_index('date')
        data_transformed.sort_values(by='date', ascending=True, inplace=True)
        data_transformed = data_transformed[~data_transformed.duplicated()]
        data_transformed = data_transformed.reset_index()

        export_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/{file}'
        data_transformed.to_csv(export_path, 
                                sep=';',
                                index=None,
                                date_format='%Y-%m-%d')

Transformation to 'device_unlocks.csv' file

In [62]:
file_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/device_unlocks.csv'
data = pd.read_csv(file_path,
                   delimiter=';',
                   parse_dates=['date'])

export_path = f'D:/Estiven/Datos/Proyectos/statistics-of-use-project/data/processed/device_unlocks.csv'
data.to_csv(export_path, 
                        sep=';',
                        index=None,
                        date_format='%Y-%m-%d')

C:\Users\User\AppData\Local\Temp\ipykernel_12776\77304124.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(file_path,
